In [1]:
import os
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [3]:
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for file in tqdm(filelist[-300:]):
    df0 = pd.read_csv(os.path.join(path, file))
    df = pd.concat([df, df0], ignore_index=True)

100%|██████████| 300/300 [00:01<00:00, 160.40it/s]


In [4]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()

In [5]:
# 计算中轨线(MB)、上轨线(UB)和下轨线(LB)
def get_mb_ub_lb(df, length=20, K=2):
    df = df.copy()
    df['MB'] = df['c'].rolling(length).mean()  # 计算20日收盘价的简单移动平均线
    # 计算标准差
    df['seta'] = np.sqrt((np.square(df['c'] - df['MB'])).rolling(length).mean())
    df['UB'] = df['MB'] + K * df['seta']
    df['LB'] = df['MB'] - K * df['seta']
    df['g1.BBW'] = (df['UB'] - df['LB']) / df['MB']
    return df['g1.BBW']

In [6]:
df1['g1.BBW'] = get_mb_ub_lb(df1)
df1 = df1[39:]
df1

,t,o,h,l,c,v,g1.BBW
39,1671116400000,17448.0,17483.2,17355.0,17424.0,30874.038,0.036998
40,1671120000000,17424.0,17437.2,17330.0,17391.7,26907.949,0.039019
41,1671123600000,17391.7,17424.4,17367.2,17392.1,15494.334,0.040952
42,1671127200000,17392.1,17415.2,17310.0,17395.5,19325.547,0.042449
43,1671130800000,17395.5,17467.7,17383.0,17435.1,17252.568,0.043105
...,...,...,...,...,...,...,...
7195,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,0.030692
7196,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,0.030789
7197,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,0.030729
7198,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,0.030741


In [7]:
df1.to_csv('BBWH.csv', index=False)